In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
campaign = pd.read_csv("../data/Campaign.csv", encoding="latin-1")
profile_creation = pd.read_csv("../data/EPO_Teradata_Employer_Profile_Creation_Report.csv")
job_seeker = pd.read_csv("../data/EPO_Teradata_Job Seeker_Profile_Creation_Report.csv")
job_board = pd.read_csv("../data/EPO_Teradata_Job_Board_Sales_Report.csv")
feedback = pd.read_csv("../data/Feedback__c.csv", encoding="latin-1", low_memory=False)
account = pd.read_csv("../data/SalesForce_Account.csv", encoding="latin-1", low_memory=False)
case = pd.read_csv("../data/SalesForce_Case.csv", encoding="latin-1", low_memory=False)
info_c = pd.read_csv("../data/SalesForce_Hire_Information__c.csv", encoding="latin-1")
opp = pd.read_csv("../data/SalesForce_Opportunity.csv", low_memory=False)
record = pd.read_csv("../data/SalesForce_RecordType.csv")

#no columns to pull data from
#sales_2018 = pd.read_csv("../data/SalesForce_2018Activities.csv")
#sales_force = pd.read_csv("../data/SalesForce_Contact.csv")
#email = pd.read_csv("../data/vr__VR_Email_History_Contact__c.csv")


In [3]:
ls = [campaign, profile_creation, job_seeker, job_board, feedback, account, case, 
      info_c, opp, record]
ls_name = ["campaign", "profile_creation", "job_seeker", "job_board", "feedback", "account", "case", 
      "info_c", "opp", "record"]

df_dict = dict(zip(ls_name, ls))
print("---Dataframes Shapes---")
for x in range(len(df_dict)):
    print(f"{list(df_dict.keys())[x]} Shape: {list(df_dict.values())[x].shape}")   

---Dataframes Shapes---
campaign Shape: (1386, 103)
profile_creation Shape: (778, 5)
job_seeker Shape: (4342, 5)
job_board Shape: (521, 8)
feedback Shape: (15807, 70)
account Shape: (16858, 168)
case Shape: (14845, 56)
info_c Shape: (30754, 34)
opp Shape: (10849, 130)
record Shape: (80, 13)


In [47]:
def df_sql(df_ls,name, conn):
    
    """
    Load all data into a sqlite3 DB
    """
    
    conn = sqlite3.Connection("../data/heroes.sqlite3")
    
    for x in range(len(df_ls)):
        df_ls[x].to_sql(f"{name[x]}", conn, index=False)
    
    conn.close()
        
    return   

In [48]:
# Load the DB
df_sql(ls, ls_name, conn)

In [21]:
# Extract CREATE TABLE statements

conn = sqlite3.Connection("../data/heroes.sqlite3")
curs = conn.cursor()

query = curs.execute("SELECT * FROM sqlite_master WHERE type='table'").fetchall()
table_ls = []

for x in range(len(query)):
    table_ls.append(query[x][4].replace("\n", "") + ";")